## Clean Functions

In [1]:
// Lets say we have our small little lovely in-memory awesome database!!

// Here are our tables:
case class User(id: Int, name: String, email: Option[String], country: Option[String])
case class Discount(name: String, percent: Double)
case class Revenue(user: User, product:String, totalBilled: Double)  // Never use Double for prices!



def getUserById(id: Int): Option[User] = {
  // The mad database!
  if (id % 13 == 0){
    None // GRPR anonymized the unlucky
  } else if(id % 2 == 0) {
    Some(User(id, "SpEC", None, None)) // You signing up on the SpEC'19 app
  } else {
    Some(User(id, "A real name", Some("fname@lname.com"), Some("SE"))) // You on Spotify
  }
}

def getPriceByProduct(product: String, country: String): Option[Double] = {
  Map(
    ("Premium", "US") ->  9.99,
    ("Family",  "US") -> 14.99,
    ("Premium", "SE") ->  79.9,
    ("Family",  "SE") -> 139.9,
  ).get( (product, country) )
}

def getDiscountByCountry(country: String): Option[Discount] = {
  // This is our database
  val discountsByCountry: Map[String, Discount] =
    Map(
      "US" -> Discount("us-student", 50),
      "SE" -> Discount("se-student", 40),
      "DE" -> Discount("de-student", 45)
    )
  discountsByCountry.get(country)
}



defined class User
defined class Discount
defined class Revenue
defined function getUserById
defined function getPriceByProduct
defined function getDiscountByCountry

In [2]:
// Expected
def billableAmount: (Int, String) => Option[Double] = {
  case (userId, product) =>
    for {
      user     <- getUserById(userId)
      _        =  println(s"User is ${user.name}, from ${user.country}") // Assignment / println returns Unit
      country  <- user.country
      price    <- getPriceByProduct(product, country)
      _        =  println(s"The price of $product in $country is $price")
      discount <- getDiscountByCountry(country)
      _        =  println(s"The discount available for $country is ${discount.percent}")
    } yield price - (price * discount.percent / 100)
}

defined function billableAmount

In [3]:
billableAmount(1, "Premium")

User is A real name, from Some(SE)
The price of Premium in SE is 79.9
The discount available for SE is 40.0


res2: Option[Double] = Some(47.940000000000005)

In [4]:
billableAmount(2, "Premium")

User is SpEC, from None


res3: Option[Double] = None

In [5]:
billableAmount(13, "Premium")

res4: Option[Double] = None

We now have clean functions.

In [6]:
// Lets remove debugs and take a look at a clean function composition.
// Expected
def billableAmount: (Int, String) => Option[Double] = {
  case (userId, product) =>
    for {
      user     <- getUserById(userId)
      country  <- user.country
      price    <- getPriceByProduct(product, country)
      discount <- getDiscountByCountry(country)
    } yield price - (price * discount.percent / 100)
}

defined function billableAmount

In [7]:
// Expected
def billableAmount: (Int, String) => Option[Double] = {
  case (userId, product) =>
    getUserById(userId)
        .flatMap{
            user => 
                user.country.flatMap{
                    country =>
                        getDiscountByCountry(country).flatMap{
                            discount =>
                                getPriceByProduct(product, country).map{
                                    price =>
                                        price - (price * discount.percent / 100)
                                }
                        }
                         
                }
        }
}

defined function billableAmount

In [8]:
billableAmount(1, "Premium")

res7: Option[Double] = Some(47.940000000000005)

In [9]:
billableAmount(2, "Premium")

res8: Option[Double] = None

In [10]:
billableAmount(13, "Premium")

res9: Option[Double] = None